[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/activities/02_ode_review/ode_review.ipynb)

In [ ]:
# Necessity libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from math import exp, cos, sin

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# ODE review workshop

## A Predator–Prey ODE Model

A two-variable nonlinear ODE, the Lotka–Volterra equation, can be used to model populations of two species:
$$
y' =
\left[
\begin{array}{c}
 y_1(\alpha_1 - \beta_1 y_2) \\
 y_2(-\alpha_2 + \beta_2 y_1)
\end{array}
\right]
\equiv f(y).
$$

The $\alpha$ and $\beta$ are modeling parameters, describe birth rates, death rates, predator--prey interactions.

In [ ]:
# Constants in model
alpha1 = 1.2
beta1 = 0.4
alpha2 = 0.2
beta2 = 0.1

# Function that evaluates the RHS of the ODE. It has two components,
# representing the changes in prey and predator populations.
def f_odeint(y, t):
    return np.array([alpha1 * y[0] - beta1 * y[0] * y[1],
                     -alpha2 * y[1] + beta2 * y[0] * y[1]])

def f_solve_ivp(t, y):
    return np.array([alpha1 * y[0] - beta1 * y[0] * y[1],
                     -alpha2 * y[1] + beta2 * y[0] * y[1]])


# Specify the range of time values where the ODE should be solved at
time = np.linspace(0, 70, 500)

# Initial conditions, set to the initial populations of prey and predators
yinit = np.array([10, 5])

# Solve ODE using the "odeint" library in SciPy
# y = odeint(f_odeint, yinit, time) # deprecated

# Solve ODE using the "solve_ivp" library in SciPy
sol = solve_ivp(f_solve_ivp, [time[0], time[-1]], yinit, t_eval=time)
y = sol.y.T

# Print the solutions
# for i in range(0,500):
#    print(time[i],x[i,0],x[i,1])

# Plot the solutions
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=300)
p0, = ax.plot(time, y[:, 0], label="Prey")
p1, = ax.plot(time, y[:, 1], label="Predators")
ax.set_xlabel('$t$')
ax.set_ylabel('Population')
ax.legend()
plt.tight_layout()
plt.show()

## Euler's method for $y' = \lambda y$

In [ ]:
# Initial variables and constants
y = 1
t = 0
h = 0.1
lam = 0.5

# Store the results
t_values = []
y_values = []
y_exact_values = []

# Apply Euler step until t>2
while t <= 2:

    # Analytical solution
    yexact = exp(lam*t)

    # Print the solutions and error
    print(t, y, yexact, y-yexact)
    # Store the results
    t_values.append(t)
    y_values.append(y)
    y_exact_values.append(yexact)

    # Euler step
    y = y+h*(lam*y)

    # Update time
    t += h


In [ ]:
# Plot the numerical and exact solutions
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=300)

ax.plot(t_values, y_values, label='Numerical (Euler)', marker='o', linestyle='-', markersize=4)
ax.plot(t_values, y_exact_values, label='Exact', linestyle='--', lw=2)

# Formatting
ax.set_xlabel('$t$')
ax.set_ylabel('$y$')
ax.legend()
ax.grid()

plt.show()

## Euler's method stability check

In [ ]:
# Initial variables and constants
y = 1
t = 0
h = 0.1

# Choose the constant in the ODE, dy/dt=-lam*y. We need -2=<h*lam=<0 for stability.
lam = -21

# Store the results
t_values = []
y_values = []
y_exact_values = []

# Apply forward Euler step until t>1
while t <= 1:

    # Analytical solution
    yexact = exp(lam*t)

    # Print the solutions and error
    print(t, y, yexact, y-yexact)
    # Store the results
    t_values.append(t)
    y_values.append(y)
    y_exact_values.append(yexact)

    # Euler step
    y = y+h*(lam*y)

    # Update time
    t += h

In [ ]:
# Plot the numerical and exact solutions
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=300)

ax.plot(t_values, y_values, label='Numerical (Euler)', marker='o', linestyle='-', markersize=4)
ax.plot(t_values, y_exact_values, label='Exact', linestyle='--', lw=2)

# Formatting
ax.set_xlabel('$t$')
ax.set_ylabel('$y$')
ax.legend()
ax.grid()

plt.show()

## Second-order Runge–Kutta methods

Three such examples are:

- The modified Euler method ($a=0$, $b=1$, $\alpha = \beta = 1/2$): $$y_{k+1} = y_k + hf\left(t_k +  \frac{1}{2}h, y_k + \frac{1}{2}hf(t_k,y_k)\right).$$
- The improved Euler method (or Heun's method) ($a=b=1/2$, $\alpha=\beta=1$): $$y_{k+1} = y_k + \frac{1}{2}h[f(t_k,y_k) + f(t_k+h,y_k+hf(t_k,y_k))].$$
- Ralston's method ($a=1/4$, $b=3/4$, $\alpha=2/3$, $\beta=2/3$) $$y_{k+1} = y_k + \frac{1}{4}h[f(t_k,y_k) + 3f(t_k+\tfrac{2h}{3},y_k+\tfrac{2h}{3}f(t_k,y_k))].$$


In [ ]:
# Initial variables and constants
t = 0
h = 0.05

# Function to integrate
def f(t, y):
    return exp(-0.5*t*t)-y*t


# Starting values for modified Euler, improved Euler, and Ralston
yme = 0
yie = 0
yre = 0

# Store the results
t_values = []
y_me_values = []
y_ie_values = []
y_re_values = []
y_exact_values = []

# Apply timesteps until t>6
while t <= 6:

    # Analytical solution
    yexact = t*exp(-0.5*t*t)

    # Print the solutions and error
    print(t, yme, yie, yre, yexact, yme-yexact, yie-yexact, yre-yexact)
    # Store the results
    t_values.append(t)
    y_me_values.append(yme)
    y_ie_values.append(yie)
    y_re_values.append(yre)
    y_exact_values.append(yexact)

    # Modified Euler step
    k1 = h*f(t, yme)
    k2 = h*f(t+0.5*h, yme+0.5*k1)
    yme += k2

    # Improved Euler step
    k1 = h*f(t, yie)
    k2 = h*f(t+h, yie+k1)
    yie += 0.5*(k1+k2)

    # Ralston's method
    k1 = h*f(t, yre)
    k2 = h*f(t+2*h/3., yre+k1*2/3.)
    yre += 0.25*k1+0.75*k2

    # Update time
    t += h

In [ ]:
# Create a single subplot
fig, ax1 = plt.subplots(1, 1, figsize=(8, 4), dpi=300)

# Plot the results
ax1.plot(t_values, y_me_values, label='Modified Euler', marker='o', linestyle='-', markersize=4)
ax1.plot(t_values, y_ie_values, label='Improved Euler', marker='s', linestyle='-', markersize=4)
ax1.plot(t_values, y_re_values, label="Ralston's Method", marker='^', linestyle='-', markersize=4)
ax1.plot(t_values, y_exact_values, label='Exact', linestyle='--', lw=2)


# Formatting
ax1.set_xlabel('$t$')
ax1.set_ylabel('$y$ and Error')
ax1.legend()
ax1.grid()

plt.show()

## Fourth-order Nystorm method

The example implements the Nystrom method using the Butcher tableau
$$
  \begin{array}{c|ccc}
  0 & & & \\
  \tfrac{1}{2} & \tfrac{1}{8} & & \\
  1 & 0 & \tfrac{1}{2} & \\
  \hline
  \bar{\gamma}_i & \tfrac{1}{6} & \tfrac{1}{3} & 0 \\
  \hline
  \gamma_i & \tfrac{1}{6} & \tfrac{2}{3} & \tfrac{1}{6}
  \end{array}
$$

Here, the final two lines give the two sets of coefficients required to update $y$ and $y'$.

In [ ]:
# Function to integrate
def f(t, y):
    return -y


# Integration interval
L = 10


def nystrom(N, output):

    # Initial values for y(0) and y'(0)
    y = 0
    dy = 1

    # Number of timesteps, and step size
    t = 0
    h = L/N
    h2 = h*h

    # Apply Euler step until t>2
    if output:
        print(0, y, dy)
    for i in range(N):

        # Nystrom intermediate steps
        t = i*h
        dk1 = f(t, y)
        dk2 = f(t+0.5*h, y+0.5*h*dy+h2*0.125*dk1)
        dk3 = f(t+h, y + h*dy+h2 * 0.5*dk2)

        # Update y and its derivative
        y += h*dy+h2/6*(dk1+2*dk2)
        dy += h/6*(dk1+4*dk2+dk3)

        if output:
            print(t+h, y, dy)

    return (y, dy)


# Run the integrator once and print out the solution
nystrom(200, True)
# sys.exit()

# Store the results
results = []

# Run the integrator many times to see how the error scales
for N in (16, 32, 64, 128, 256, 512):
    (y, dy) = nystrom(N, False)
    print(N, L/N, y-sin(L), dy-cos(L))
    results.append((N, L/N, y-sin(L), dy-cos(L)))

In [ ]:
# Extract step sizes and errors from the results
step_sizes = [result[1] for result in results]
errors = [abs(result[2]) for result in results]  # Use the error in y

# Plot the errors on a log-log scale
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=300)

ax.loglog(step_sizes, errors, marker='o', linestyle='-', label='Abs. error in $y$')

# Add a reference line with slope -4
ref_slope = [errors[0] * (step_sizes[0] / h) ** (-4) for h in step_sizes]
ax.loglog(step_sizes, ref_slope, linestyle='--', label='Reference slope $O(h^{4})$')

# Formatting
ax.set_xlabel('Step size ($h$)')
ax.set_ylabel('Error')
ax.legend()

plt.show()